In [1]:
# install Amumo

# need ipykernel > 6
# ! pip install ipykernel==6.23.1
# ! pip install git+https://github.com/ginihumer/Amumo.git

In [2]:
# import 
from amumo import model as am_model
from amumo import data as am_data
from amumo import widgets as am_widgets

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

# Basic usage

In [3]:
# load dataset
dataset = am_data.DiffusionDB_Dataset(path="2m_first_1k", batch_size=100) # data helper for the diffusionDB dataset; for the interactive prototype, we only use a random subset of 100 samples
all_images, all_prompts = dataset.get_data()
cache_name = 'diffusiondb_random_100' # path used to cache the results

Found cached dataset diffusiondb (C:/Users/Christina/.cache/huggingface/datasets/poloclub___diffusiondb/2m_first_1k/0.9.1/b3bc1e64570dc7149af62c4bac49ecfbce16b683dd4fee083292fae1afa95f7c)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# This widget consists of a similarity heatmap that shows the similarities between all latent embeddings, a 2-dimensional projection of the embeddings, and a hover view of the currently hovered instance.
# It also allows user input for choosing the model, an option to manually close the modality gap, an option to cluster the matrix rows by their similarity, and settings for the 2-dimensional projection. 
# To learn more about the proper use for closing the modality gap, see the interactive article: https://jku-vds-lab.at/amumo.
# You can specify a list of models to compare; there is a predefined set of models, but you can also pass your own model class.
# The widget takes a list of images and a list of prompts of the same size as input. Images and texts with the same index are considered pairs.
am_widgets.CLIPExplorerWidget(cache_name, all_images, all_prompts, models=[am_model.CLIPModel(), 'CyCLIP', 'CLOOB_LAION400M']) 

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:356: FutureWarning: The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:356: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_images)


found cached embeddings for diffusiondb_random_100_CLIP_RN50


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('CLIP', 'C…

In [5]:
# This widget shows similarity heatmaps for all models in the list for a better comparison between models.
am_widgets.CLIPComparerWidget(cache_name, all_images, all_prompts, models=[am_model.CLIPModel(), 'CyCLIP', 'CLOOB_LAION400M']) 

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:534: FutureWarning:

The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:534: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for diffusiondb_random_100_CLIP_RN50
found cached embeddings for diffusiondb_random_100_CyCLIP_RN50
batch 1 of 1


CLIPComparerWidget(children=(HoverWidget(children=(VBox(children=(HTML(value='', layout=Layout(width='300px'))…

# Use a Custom Model

In [6]:
import clip
import torch
import numpy as np
# You can use the CLIPModelInterface to create your own model class wrapper.
class CustomModel(am_model.CLIPModelInterface):
    available_models = clip.available_models()
    model_name = 'MyCLIP'

    def __init__(self, name='RN50', device='cpu') -> None:
        super().__init__(name, device)
        self.model, self.preprocess = clip.load(name, device=device)
        self.model.eval()
        self.logit_scale = self.model.logit_scale

    def encode_image(self, images):
        images = [self.preprocess(i) for i in images]
        image_input = torch.tensor(np.stack(images)).to(self.device)
        return self.model.encode_image(image_input).float().cpu()

    def encode_text(self, texts):
        text_tokens = clip.tokenize(texts, truncate = True).to(self.device)
        return self.model.encode_text(text_tokens).float().cpu()

In [7]:
# The widget takes any instance of type CLIPModelInterface.
am_widgets.CLIPExplorerWidget(cache_name, all_images, all_prompts, models=[CustomModel()])

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:356: FutureWarning:

The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:356: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for diffusiondb_random_100_MyCLIP_RN50


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('MyCLIP',)…